# Simulating Heat Diffusion to Cook a Chicken with Numba and CUDA

In this notebook, we'll attemp tolving the 2-dimensional heat equation to simulate how heat diffuses through a chicken as it cooks in the oven. 

To start, we'll use a simplified 2D representation of a chicken.

In [ ]:
import numpy as np
from numba import cuda, njit, prange
import matplotlib.pyplot as plt

# Load the 2D image representation of the chicken
image = plt.imread('chicken.png')
height, width = image.shape
plt.imshow(image)
plt.colorbar()

To simulate heat diffusion in our 2D "chicken" grid, we'll assume that:

- The oven has a width of 50 cm.
- The oven temperature is set to 200°C.
- The chicken's initial internal temperature is 20°C.
- Heat diffusion across the chicken will be governed by the 2D heat equation.

Given the temperature field $u = u(x, y, z; t)$, the heat equation describing the change over time is:
$$
\frac{\partial u}{\partial t} = \alpha \nabla^2 u 
$$

where:
- $\alpha$ is the thermal diffusivity of the chicken, in units of $m^2/s$.
  
For chicken, $\alpha$ has been measured (!) to be approximately $1.32 \times 10^{-7} \, \text{m}^2/\text{s} $, allowing us to model how heat penetrates and warms the chicken over time.

To simulate heat transfer over time, we'll need to update the temperature of each "pixel" representing the chicken at each time step.

### Simulation Parameters
- Total simulation time: 1 hour
- Time step: $\Delta t = 1 \, \text{s}$

### Discretizing the Heat Equation

The continuous heat equation can be expressed as:
$$
\frac{\partial u}{\partial t} = \alpha \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right)
$$

To implement this, we discretize the equation in both time and space.

Let $u_{i,j}^{\tau}$ represent the temperature at pixel $(i, j)$ at time $\tau$. The discrete form of the equation then becomes:

$$
\frac{u_{i,j}^{\tau+1} - u_{i,j}^{\tau}}{\Delta t} = \alpha \left( \frac{u_{i+1,j}^{\tau} - 2u_{i,j}^{\tau} + u_{i-1,j}^{\tau}}{\Delta x^2} + \frac{u_{i,j+1}^{\tau} - 2u_{i,j}^{\tau} + u_{i,j-1}^{\tau}}{\Delta y^2} \right)
$$

This formulation allows us to iteratively update each pixel's temperature at every time step.

To update the temperature field at the next time step $\tau+1$, we use the current state of the temperature field at time $\tau$.

Assuming that each pixel represents an equal spatial step size along both $x(i)$ and $y(j)$ axes, we set $\Delta x = \Delta y = \Delta s$.

The update rule for each pixel $(i, j)$ at time $\tau+1$ then becomes:

$$
u_{i,j}^{\tau+1} = u_{i,j}^{\tau} + \alpha \frac{\Delta t}{\Delta s^2} \left( u_{i+1,j}^{\tau} + u_{i-1,j}^{\tau} + u_{i,j+1}^{\tau} + u_{i,j-1}^{\tau} - 4u_{i,j}^{\tau} \right)
$$

This equation allows us to iteratively compute the temperature at each pixel for the next time step, based on its current state and the temperatures of its neighboring pixels.

This is finally the equation we would like to solve for all pixels of our chicken and for all timesteps.

In [ ]:
# Create a mask to identify the "chicken" pixels within the image
# Here, pixels with values less than 0.9 are considered part of the chicken
mask = image < 0.9

# Display the mask to verify which pixels belong to the chicken
plt.imshow(mask, cmap='gray')
plt.title("Chicken Pixel Mask")
plt.show()

In [ ]:
# Initialize the temperature field
T_oven = 200.0  # Oven temperature in degrees Celsius
T_chicken = 20.0  # Initial chicken temperature in degrees Celsius

# Create a 2D array representing the temperature field, setting all pixels initially to the oven temperature
T_initial = np.full((height, width), T_oven, dtype=np.float32)

# Set the initial temperature of the chicken pixels based on the mask
T_initial[mask] = T_chicken

# Display the initial temperature distribution
plt.imshow(T_initial, cmap='coolwarm', vmin=T_chicken, vmax=T_oven)
plt.colorbar(label="Temperature (°C)")
plt.title("Initial Temperature Field of Chicken in Oven")
plt.show()

In [6]:
# Parameters for the heat equation
oven_width = 0.5                # Width of the oven in meters
alpha = 1.32e-7                 # Thermal diffusivity of chicken in m^2/s
dx = oven_width / width         # Spatial step size in meters (based on pixel size)
dt = 1.0                        # Time step size in seconds
time = 3600.0                   # Total cooking time in seconds (1 hour)
num_steps = int(time / dt)      # Total number of time steps

# Precompute constant factor for the heat equation to simplify computation
dt_dx2 = dt / (dx ** 2)         # This factor is used frequently in the update equation

## Numba CPU implementation

Write the purely-CPU implementation of the task using Numba

In [7]:
# Numba heat equation function on CPU
@njit(parallel=True)
def heat_equation_cpu(Temp, Mask, Alpha, Dt_dx2, Num_steps):
    """
    Simulates heat diffusion over time on the CPU using Numba.

    Parameters:
    - Temp: 2D array representing the temperature field.
    - Mask: 2D boolean array indicating chicken pixels.
    - Alpha: Thermal diffusivity of the material (m^2/s).
    - Dt_dx2: Precomputed constant value for the heat equation.
    - Num_steps: Total number of time steps to simulate.

    Returns:
    - Updated temperature field after simulation.
    """

    ...

    return Temp

In [8]:
# Run the simulation on the CPU
T_final = heat_equation_cpu(T_initial, mask, alpha, dt_dx2, num_steps)

In [ ]:
# Display the final temperature distribution after simulation
plt.imshow(T_final, cmap='coolwarm', vmin=T_chicken, vmax=T_oven)
plt.colorbar(label="Temperature (°C)")
plt.title("Final Temperature Distribution of the Chicken After Cooking")
plt.show()

## Numba CUDA implementation

Write the equivalent implementation for the GPU using Numba and CUDA.

There are several possible implementation that could be tested.
Some of them will be particularly problematic due to the way threads in a GPU are executed.

Try to implement first the following configuration:
- Write one single CUDA kernel that performs the evolution over all time steps  
- Use a 2D grid of threads per block (max dimension `(32,32)` threads per block due to HW limitation)
- Decide if (and where, if this is the case) to implement thread synchronization 

**WARNING** It's possible that in the case of naive implementations, the kernel run time exceeds 5s. This is however set as an hard limit by the Jetson Nano due to its inherent limitations, and the resulting application could thus fail with an error similar to:

```c
CudaAPIError: [702] Call to cuCtxSynchronize results in CUDA_ERROR_LAUNCH_TIMEOUT
```

Consider disabling the timeout by using the script provided under `/MCP/disable_timeout.sh`
```bash
sudo ./MCP/disable_timeout.sh
```

In [10]:
@cuda.jit
def heat_equation_gpu(...):

    ...

In [11]:
# Transfer data to the GPU


In [12]:
# Configure the thread grid for CUDA kernel launch


In [13]:
num_steps = np.int32(time)  # Total number of time steps for the simulation

# Launch the CUDA kernel on the GPU
heat_equation_gpu[blocks_per_grid, threads_per_block](...)

In [ ]:
# Copy the final temperature distribution back to the host
T_final = ...

# Display the final temperature distribution
plt.imshow(T_final, cmap='coolwarm', vmin=T_chicken, vmax=T_oven)
plt.colorbar(label="Temperature (°C)")
plt.title("Final Temperature Distribution of the Chicken After Cooking (GPU)")
plt.show()

In [ ]:
# Display the final temperature distribution - With grids sized as the thread blocks
plt.imshow(T_final, cmap='coolwarm', vmin=T_chicken, vmax=T_oven)

ax = plt.gca()

for x in range(-1, image.shape[1], BLOCK_SIZE):
    ax.axvline(x=x, color='grey', linestyle='-', linewidth=0.5)
for y in range(-1, image.shape[0], BLOCK_SIZE):
    ax.axhline(y=y, color='grey', linestyle='-', linewidth=0.5)

plt.xlim(0,T_final.shape[0])
plt.ylim(T_final.shape[1],0)

plt.colorbar()

## Numba CUDA Alternative Implementation

In the naive CUDA implementation, all timesteps are processed within a single kernel launch, with each thread performing the update calculations for each time step and synchronizing after each step. This approach leads to several potential issues, one of the most visible ones being the boundary conditions of each block acting as the "limit" for the heat exchange computation.

In this case, try to implement alternative solutions for this problem, by reverting to a simpler approach:
- Write a simpler kernel to perform individual timesteps on the device
- Loop over the timesteps at host-side and execute the kernel to produce the new Temperature map
- Synchronize the device before performing the next step with `cuda.synchronize()`
- Swap the _new_ and _old_ Temperature maps to avoid allocating new memory

In [21]:
# Numba CUDA kernel function for a single time step of the heat equation
@cuda.jit
def heat_equation_gpu_1step(...):

    ...

In [22]:
# Transfer data to the GPU


In [23]:
# Loop over time steps, updating the temperature distribution
for step in range(num_steps):
    # Launch the kernel for a single time step
    heat_equation_gpu_1step[blocks_per_grid, threads_per_block](...)    
    
    # Synchronize the device to ensure the kernel execution is complete
    cuda.synchronize()
    
    # Swap the new and previous temperature matrices for the next time step


In [ ]:
# Copy the final temperature distribution back to the host
T_final = ...

# Display the final temperature distribution
plt.imshow(T_final, cmap='coolwarm', vmin=T_chicken, vmax=T_oven)
plt.colorbar(label="Temperature (°C)")
plt.title("Final Temperature Distribution of the Chicken After Cooking (GPU)")
plt.show()